# Submitting a model

This notebook will guide you through the model submission process. To start, if you're in the notebook view page, click "... -> Copy and Edit" on the right.


## Step 0: Install dependencies

We need to install our dependencies here. Just run the following cell:

In [ ]:
!pip install ../input/dependencies/dotmap-1.3.23-py3-none-any.whl

## Step 1: Make a submission package

First, save the model checkpoint you want to submit under `attachments`, along with any other files that will be loaded during test time. That is in the repository root:

```bash
(base) poesia@node01-ccncluster:/data/poesia/riiid/ednet-challenge$ cp path/to/model.ckpt attachments/
```

Then, run:

```bash
(base) poesia@node01-ccncluster:/data/poesia/riiid/ednet-challenge$ ./make_submission_package.sh
```

This will create a file called `submission.tar.gz`.

## Step 2: Upload the submission package as data in this notebook

If the submission package is on CCN, first you'll need to get it on your local machine. `scp` will do the trick.

Having the package, on the right side of this screen, click on "Add data", and then "Upload". Under name, put "Submission N", where N is a number. It can be any number, but each time you submit it needs to be different. Then, upload `submission.tar.gz` and click on "Create". Kaggle will take a few seconds to upload it and then download it here in the notebook.

If all went well, on the right, inside the `input` folder, you'll see `submission-n` (this might take a minute - go drink water). In the cell below, change `n` and run it.

In [ ]:
SUBMISSION_DIR = 'submission-4'

## Step 3: Set up the Python Path

Kaggle has automatically unpacked `submission.tar.gz` under `/kaggle/input`. Now, we'll just add that to our Python Path, so that we can use code from the repository. Additionally, we'll do the same for a Kaggle-specific package with the submission API. For that, just run the cell below.

In [ ]:
import sys

sys.path.append('/kaggle/input/riiid-test-answer-prediction')
sys.path.append('/kaggle/input/' + SUBMISSION_DIR)

## Step 4: Initialize the test environment

Just run the cell below. You can run this only once per notebook run; to run it again, you need to restart the notebook entirely.

In [ ]:
import riiideducation
env = riiideducation.make_env()
iter_test = env.iter_test()

## Step 5: Load your model

This part might need some work if it's the first time you submit a model with this kind of agent. We now need to load the model in a wrapper that unifies the API and makes it easy to submit it. In the repository, the wrapper interface is located in `ednet/common/submission.py`. Each agent needs to implement a class that has that interface. The example below loads a DKTAgent. This part will be model-specific, but shouldn't be too different from DKT.

Also, you probably want to load your model on a GPU (or TPU if you're fancy). To make one available, enable "Accelerator" on the right side of the screen, and properly load your model on that device.

In [ ]:
from ednet.dkt.agents.dkt import DKTSubmissionWrapper
agent = DKTSubmissionWrapper(f'/kaggle/input/{SUBMISSION_DIR}/attachments/dkt-test.ckpt')

## Step 6: Run on the test set!

Finally! Just run the cell below to run your model on the test set.

Important: note that you almost certainly do not want to pass empty dicts to the last parameters here. Instead, you'll want to save them from the dataset after the model trains (probably as a pickle file), attach that and load it.

In [ ]:
import pandas as pd
pd.options.mode.chained_assignment = None

from ednet.common.submission import TestSetRunner

runner = TestSetRunner(agent,
                       f'/kaggle/input/{SUBMISSION_DIR}/attachments/cache_temporal_feat_splits_train_250k_valid_25k_cv1.pickle',
                       iter_test,
                       {},
                       {},
                       {})
runner.run(env)

# Step 7: Submit the notebook

If all went well, you just ran your code on the example test set. Now it's time to make an actual submission.  First, **make sure Internet is turned off for the notebook** (on the right). Click "Save Version" above, give your version a name, then go to the competition's page, and "Submit predictions". Choose the right notebook, and you're done! If your model works, submission takes a considerably long time (at least 3-4 hours, even for a trivial baseline model), so go do something else and check your results later in the day or tomorrow.